In [1]:
%load_ext autoreload
%autoreload 2
%aimport

!nvidia-smi

Modules to reload:
all-except-skipped

Modules to skip:

Thu Apr  1 20:25:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 440.59       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   33C    P0    35W / 300W |   5892MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                   

- https://github.com/haqishen/Google-Landmark-Recognition-2020-3rd-Place-Solution
- https://www.kaggle.com/zzy990106/b0-bert-cv0-9
- https://github.com/yang-zhang/product_category/blob/dev/notebooks/transformer_20210307E1--pin_memory.ipynb

In [2]:
HOME = "/data/git/shopee-product-matching"
p_out = f'{HOME}/output/trn0001'
!mkdir -p {p_out}
p_prp = f'{HOME}/output/prep001/train_prep.csv'

FOLD = 0


import sys
sys.path.append(f"{HOME}/src")

import pandas as pd
from argparse import ArgumentParser
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pl_model import ShpModel, ShpDataModule
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
parser = ArgumentParser()
parser.add_argument(
    "--train_batch_size",
    help="How many samples per batch to load for train dataloader.",
    type=int,
    default=64,
)
parser.add_argument(
    "--val_batch_size",
    help="How many samples per batch to load for validation dataloader.",
    type=int,
    default=128,
)
parser.add_argument(
    "--dataloader_num_workers",
    help="How many subprocesses to use for data loading. 0 means that the data will be loaded in the main process.",
    type=int,
    default=8,
)
parser.add_argument(
    "--pin_memory",
    help="Wether to use pin_memory in pytorch dataloader. If True, the data loader will copy Tensors into CUDA pinned memory before returning them.",
    action="store_true",
)

parser = pl.Trainer.add_argparse_args(parser)
parser = ShpModel.add_model_specific_args(parser)
args_list = [
    '--default_root_dir', p_out,
    '--kernel-type', '',
    '--enet-type', 'tf_efficientnet_b1_ns',

]

args = parser.parse_args(args_list)

In [4]:
df = pd.read_csv(p_prp)
display(df.head(3))
dftrn = df[df.fold!=FOLD].copy()
dfval = df[df.fold==FOLD].copy()
dftrn.shape, dfval.shape

data_module = ShpDataModule(
    dftrn=dftrn,
    dfval=dfval,
    train_batch_size=args.train_batch_size,
    val_batch_size=args.val_batch_size,
    pin_memory=args.pin_memory,    
    
)

posting_id                                 image       image_phash  \
0   train_129225211  0000a68812bc7e98c42888dfb1c07da0.jpg  94974f937d4c2433   
1  train_3386243561  00039780dfc94d01db8676fe789ecd05.jpg  af3f9460c2838f0f   
2  train_2288590299  000a190fdd715a2a36faed16e2c65df7.jpg  b94cb00ed3e50f78   

                                               title  label_group  fold  \
0                          Paper Bag Victoria Secret    249114794     0   
1  Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...   2937985045     2   
2        Maling TTS Canned Pork Luncheon Meat 397 gr   2395904891     0   

                                            filepath  
0  /data/git/shopee-product-matching/input/shopee...  
1  /data/git/shopee-product-matching/input/shopee...  
2  /data/git/shopee-product-matching/input/shopee...

In [5]:
data_module.setup()
data_module.num_classes

11014

In [6]:
shpmodel = ShpModel(
    kernel_type=args.kernel_type,
    enet_type=args.enet_type,
    learning_rate=args.learning_rate,
    num_classes=data_module.num_classes,
    margins=data_module.margins,
)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b1_ns-99dd0c41.pth" to /home/ubuntu/.cache/torch/hub/checkpoints/tf_efficientnet_b1_ns-99dd0c41.pth


In [7]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import CSVLogger, TensorBoardLogger

csv_logger = CSVLogger(p_out, name='csv')
tb_logger = TensorBoardLogger(p_out, name='tensorboard')

trainer = pl.Trainer.from_argparse_args(args, 
#                                         limit_train_batches=10, limit_val_batches=2, 
#                                         fast_dev_run=True,
                                        max_epochs=50,
                                        callbacks=[EarlyStopping(monitor='valid_loss')],
                                        stochastic_weight_avg=True,
                                        log_gpu_memory=True, 
                                        gpus=1,
                                        logger=[tb_logger,csv_logger],
                                       )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [8]:
pl.seed_everything(1234)
trainer.fit(shpmodel, data_module)

Global seed set to 1234
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type                      | Params
-------------------------------------------------------
0 | model    | Effnet_Landmark           | 24.1 M
1 | arc      | ArcFaceLossAdaptiveMargin | 0     
2 | accuracy | Accuracy                  | 0     
-------------------------------------------------------
24.1 M    Trainable params
0         Non-trainable params
24.1 M    Total params
96.346    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/data/anaconda3/envs/shopee/lib/python3.7/site-packages/pytorch_lightning/core/step_result.py:148: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  value = torch.tensor(value, device=device, dtype=torch.float)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

In [9]:
!find $p_out/

/data/git/shopee-product-matching/output/trn0001/
/data/git/shopee-product-matching/output/trn0001/tensorboard_csv
/data/git/shopee-product-matching/output/trn0001/tensorboard_csv/0_0
/data/git/shopee-product-matching/output/trn0001/tensorboard_csv/0_0/checkpoints
/data/git/shopee-product-matching/output/trn0001/tensorboard_csv/0_0/checkpoints/epoch=29-step=12869.ckpt
/data/git/shopee-product-matching/output/trn0001/csv
/data/git/shopee-product-matching/output/trn0001/csv/version_0
/data/git/shopee-product-matching/output/trn0001/csv/version_0/metrics.csv
/data/git/shopee-product-matching/output/trn0001/csv/version_0/hparams.yaml
/data/git/shopee-product-matching/output/trn0001/tensorboard
/data/git/shopee-product-matching/output/trn0001/tensorboard/version_0
/data/git/shopee-product-matching/output/trn0001/tensorboard/version_0/hparams.yaml
/data/git/shopee-product-matching/output/trn0001/tensorboard/version_0/events.out.tfevents.1617308772.ip-10-0-3-247.22522.0


In [10]:
!find $p_out/tensorboard/

/data/git/shopee-product-matching/output/trn0001/tensorboard/
/data/git/shopee-product-matching/output/trn0001/tensorboard/version_0
/data/git/shopee-product-matching/output/trn0001/tensorboard/version_0/hparams.yaml
/data/git/shopee-product-matching/output/trn0001/tensorboard/version_0/events.out.tfevents.1617308772.ip-10-0-3-247.22522.0


In [11]:
ls -hl {p_out}/tensorboard_csv/*/checkpoints/*.ckpt

-rw-rw-r-- 1 ubuntu ubuntu 277M Apr  1 22:18 '/data/git/shopee-product-matching/output/trn0001/tensorboard_csv/0_0/checkpoints/epoch=29-step=12869.ckpt'


In [12]:
!nvidia-smi

Thu Apr  1 22:18:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 440.59       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   46C    P0    48W / 300W |   1668MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  